In [104]:
import cv2
import glob
import numpy as np
from  Tkinter import *
import Tkinter, Tkconstants, tkFileDialog
from scipy.spatial import distance


In [105]:
#[x,y] for each right click event will be stored here
rc=list()
rc2=list()

#this function will be called whenever the mouse is right clicked
#def mouse_callback(event, x, y, flags, params):


def create_callback(img_index):
    def mouse_callback2(event2, x2, y2, flags2, params2):
    #left-click event value is 1
#     print("EVENT: %s" % event2)
        if event2==1:
            global rc    
            #store the coordinates of the right-click event
            rc.append(["point1",x2,y2,images[img_index][0]])
            #verify that the mouse click has been registered
            print (rc)
        #right-click event value is 2
        if event2==2:
            global rc2
            #store the coordinates of the right-click event
            rc2.append(["point2",x2,y2,images[img_index][0]])
            #verify that the mouse click has been registered
            print (rc2)
    return mouse_callback2

root = Tk()
root = Tk()
root.geometry("600x100")
def retrieve_input():
    global user_name
    global user_occupation
    user_name=textBox.get("1.0","end-1c")
    user_occupation=textBox1.get("1.0","end-1c")
w = Label(root, text="Hello, Please enter your name in the form: FirstName_LastName")
w1 = Label(root, text="Your Occupation")
textBox=Text(root, height=1, width=20)
textBox1=Text(root, height=1, width=20)
w.pack()
textBox.pack()
w1.pack()
textBox1.pack()
buttonCommit=Button(root, height=1, width=20, text="Save",command=lambda:retrieve_input())
buttonCommit.pack()
root.directory = tkFileDialog.askdirectory()
address=root.directory+"/*.jpg"
root.mainloop()

        
#importing all images from the folder
images = [(file, cv2.imread(file,-1)) for file in glob.glob(r"{}".format(address))]


scenes=len(images)
#opening relevent images:
for i, j in zip([5,13], [17,18]):
    
    img1 = images[i][1]
    cv2.namedWindow('image1',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image1', 640, 480)
    cv2.moveWindow('image1', 20,20)
    
    img2=images[j][1]
    cv2.namedWindow('image2',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image2', 640, 480)
    cv2.moveWindow('image2', 680,20)

    #set mouse callback function for window
    #cv2.setMouseCallback('image1', mouse_callback)
    cv2.setMouseCallback('image1', create_callback(i) )
    cv2.imshow('image1', img1)
    #cv2.setMouseCallback('image2', mouse_callback)
    cv2.setMouseCallback('image2', create_callback(j) )
    cv2.imshow('image2', img2)
    k2=cv2.waitKey(0)
    if k2==27:
        cv2.destroyWindow('image1')
    k2=cv2.waitKey(0)
    if k2==27:
        cv2.destroyWindow('image2')

[['point1', 2028, 1225, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG']]
[['point2', 2028, 1352, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG']]
[['point1', 2028, 1225, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG'], ['point1', 912, 1309, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110663.JPG']]
[['point2', 2028, 1352, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG'], ['point2', 915, 1423, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110663.JPG']]
[['point1', 2028, 1225, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG'], ['point1', 912, 1309, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110663.JPG'], ['point1', 1456, 1426, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110659.JPG']]
[['point2', 2028, 1352, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1110651.JPG'], ['point2', 915, 1423, 'C:/Users/Deepanshu/Desktop/winter school/P8/01_Cam\\P1

In [106]:
# Reading camera_v2 text file from Visual SFM output
text_file = open("cameras_v2.txt", "r")
lines = text_file.read().split('\n')

#reading header information
numberofimages=int(lines[16])
print len(lines)
text_file.close()

#Reading the image address, names, Rotation matrices(RM), Camera , Focal Length(FL) into seperate lists
imagenames=[None]*numberofimages
CameraPosition=[None]*numberofimages
R_all=[None]*numberofimages
FL=[None]*numberofimages
for i in range(numberofimages):
    imagenames[i]=lines[19+14*i]
    CameraPosition[i]=lines[23+14*i]
    R_all[i]=[lines[26+14*i],lines[27+14*i],lines[28+14*i]]
    FL[i]=lines[20+14*i]

# Storing Rotation matrices in proper float format
RM = []
tempp=[]
for i in range(len(R_all)):
    for k in range(3):
        for j in range(3):
            temporary=float((R_all[i][k].split())[j])
            tempp.append(temporary)
RM = [tempp[x:x+9] for x in range(0, len(tempp),9)]
     
#Fetching rot matrices for point one
index_rot_point1=[]
for i in range(len(rc)):
    index_rot_point1.append([index for index in range(len(imagenames)) if re.sub(r'\W+', '',imagenames[index]) == re.sub(r'\W+', '', rc[i][3])])

#Fetching Camera positions:
Camera=[]
adad=[]
for i in range(len(R_all)):
        for j in range(3):
            ad=(float(CameraPosition[i].split()[j]))
            adad.append(ad)
Camera = [adad[x:x+3] for x in range(0, len(adad),3)]

761


In [107]:
results=[]
residuals=[]
for pt in [rc,rc2]:
    RL=[] #Rotation matrices
    CX=[] #Camera positions
    fl=[] #focal lengths
    kp=[] #key points
#     residue=[] #residual
    
    # AX=B matrix form
    A=np.empty([2*len(index_rot_point1),3])
    B=np.empty([2*len(index_rot_point1),1])

    for i in range(len(index_rot_point1)):
    # RL=Rotation matrices list for the images selected
        R111=RM[int(index_rot_point1[i][0])]
        RL.append(R111)
    # CX=Camera positions list for the images selected
        X111=Camera[int(index_rot_point1[i][0])]
        CX.append(X111)
    # fl=Focal length list for the images selected
        f111=(float(FL[int(index_rot_point1[i][0])])*0.002)
        fl.append(f111)
    # Key point coordinates list    
        x1=(-(pt[i][1])+1800)*2*0.001
        y1=(-(pt[i][2])+1200)*2*0.001
        kpp=[x1,y1]
        kp.append(kpp)

    # Matrix A
        A11=(RL[i][6]*kp[i][0]+fl[i]*RL[i][0])
        A12=(RL[i][7]*kp[i][0]+fl[i]*RL[i][1])
        A13=(RL[i][8]*kp[i][0]+fl[i]*RL[i][2])
        A21=(RL[i][6]*kp[i][1]+fl[i]*RL[i][3])
        A22=(RL[i][7]*kp[i][1]+fl[i]*RL[i][4])
        A23=(RL[i][8]*kp[i][1]+fl[i]*RL[i][5])

    # MAtrix B
        B11=CX[i][0]*(RL[i][6]*kp[i][0]+fl[i]*RL[i][0])
        B12=CX[i][1]*(RL[i][7]*kp[i][0]+fl[i]*RL[i][1])
        B13=CX[i][2]*(RL[i][8]*kp[i][0]+fl[i]*RL[i][2])
        B21=CX[i][0]*(RL[i][6]*kp[i][1]+fl[i]*RL[i][3])
        B22=CX[i][1]*(RL[i][7]*kp[i][1]+fl[i]*RL[i][4])
        B23=CX[i][2]*(RL[i][8]*kp[i][1]+fl[i]*RL[i][5])

        A[i*2]=[A11,A12,A13]
        A[i*2+1]=[A21,A22,A23]
        B[i*2]=[B11+B12+B13]
        B[i*2+1]=[B21+B22+B23]

# XYZ= 3D coordinates of the two points in the local coordinate system       
    XYZ=np.linalg.lstsq(A, B)[0]
    results.append(XYZ)
    residue_temp=np.matmul(A,XYZ)
    residue=np.subtract(residue_temp,B)
    residuals.append(residue)
print(residuals)


[array([[-0.02002412],
       [-0.04375342],
       [ 0.00877028],
       [ 0.01566962],
       [ 0.04195732],
       [ 0.00143129],
       [-0.03849321],
       [ 0.02761376]]), array([[-0.01832228],
       [-0.05772567],
       [ 0.00760809],
       [ 0.01143369],
       [ 0.03477797],
       [ 0.01011896],
       [-0.03315008],
       [ 0.03697575]])]


In [108]:
dist_3d = distance.euclidean(results[0],results[1])
print(dist_3d)
act_dist=0.525
scl_fct=act_dist/dist_3d
print(scl_fct)

0.194972950808
2.69268120436


In [109]:
from laspy.file import File
from scipy.spatial import distance
import numpy as np

# Open a file in read mode:
inFile=File("CAM_Dsense - las.las")

# Finding the neighbourhood points
dataset=np.vstack([inFile.x, inFile.y, inFile.z]).transpose()
dataset

array([[-2.51676011, -0.99985403,  1.99110997],
       [-2.51255989, -0.99989402,  1.99322998],
       [-2.61422992, -1.00093997,  1.90228999],
       ..., 
       [-8.23834038,  0.729743  ,  2.52898002],
       [-8.27867031,  0.73512203,  2.54648996],
       [-8.27513982,  0.73940802,  2.54597997]])

In [110]:
# Finding all points in the range of +-p in x,y,z directions
p=1
index_pos=np.where((dataset[:,0]>=results[0][0]-p)&(dataset[:,0]<=results[0][0]+p)&(dataset[:,1]>=results[0][1]-p)&(dataset[:,1]<=results[0][1]+p)&(dataset[:,2]>=results[0][2]-p)&(dataset[:,2]<=results[0][2]+p))
# p=1
# index_pos_2=np.where((dataset[:,0]>=XYZ2[0]-p)&(dataset[:,0]<=XYZ2[0]+p)&(dataset[:,1]>=XYZ2[1]-p)&(dataset[:,1]<=XYZ2[1]+p)&(dataset[:,2]>=XYZ2[2]-p)&(dataset[:,2]<=XYZ2[2]+p))

In [111]:
# Finding the distances of all points in the neighbourhood to the point in question and finding the nearest point
dst=[]
points = []
aa=(XYZ[0][0],XYZ[1][0],XYZ[2][0])
minx = 0
miny = 0
minz = 0
min_dist =999999
mini = 0
    
for i in range(len(index_pos[0])):
     if (distance.euclidean(dataset[index_pos[0][i]],aa))<min_dist:
            min_dist = distance.euclidean(dataset[index_pos[0][i]],aa)
            minx = dataset[index_pos[0][i]][0]
            miny = dataset[index_pos[0][i]][1]
            minz = dataset[index_pos[0][i]][2]
            mini = i
                                                      
print(min_dist, minx, miny, minz, index_pos[0][mini])                 

(0.013079470378342362, -5.0970997835453717, 0.48538899438527983, 2.701639893198136, 321837)


In [112]:
# Keeping all points in 0.5 unit distance radius
pos=dataset[index_pos[0][mini]]
distances=np.sum((dataset-pos)**2, axis=1)
keep_points=distances<0.5
points_kept=inFile.points[keep_points]
print("We're keeping %i points out of %i total"%(len(points_kept), len(inFile)))

We're keeping 3727 points out of 691774 total


In [113]:
# Creating a webpage and naming it according to the user_name
trimmedLAS=user_name+'.las'
htmlDOC=user_name+'.html'
print(htmlDOC)
outFile=File(trimmedLAS, mode="w", header=inFile.header)
outFile.points=points_kept
outFile.close()

matthias_schaub2.html


In [114]:
# Creating a text file log of all variables and observations
save_path = 'C:/CrowdData/'
name_of_file = user_name
completeName = save_path + name_of_file+".txt"    
file1 = open(completeName, "w")
toFile = ("\bUser Name:\b \n" + str(user_name) + "\n\bUser Occupation:\b \n" + str(user_occupation) + "\n\bPoint1 Data:\b0 \n" + str(rc) + "\n\bPoint2 Data:\b \n" + str(rc2) + "\n\b3D Point1:\b \n" + str(results[0]) +"\n\b3D Point2:\b \n"+ str(results[1]) + "\n\b3D distance in point cloud:\b \n" + str(dist_3d) + "\n\bActual distance:\b \n" + str(act_dist) + "\n\bScaling Factor:\b \n" + str(scl_fct)) 
file1.write(toFile)
file1.close()


In [115]:
# Running potreeconverster CMD command from jupyter
import os
os.popen("C:\PotreeConverter_1.5_windows_x64\PotreeConverter.exe C:\jupyterNotebook\{0} -o C:/xampp/htdocs/potree --generate-page {1}".format(trimmedLAS,user_name))


<open file 'C:\\PotreeConverter_1.5_windows_x64\\PotreeConverter.exe C:\\jupyterNotebook\\matthias_schaub2.las -o C:/xampp/htdocs/potree --generate-page matthias_schaub2', mode 'r' at 0x0000000060E4F930>

In [116]:
# Creating the local potree webpage
html_address=str("http://localhost/potree/{0}".format(htmlDOC))
print(html_address)
# *** PLEASE CLICK ON THE LINK BELOW ***

http://localhost/potree/matthias_schaub2.html
